# Ejemplo de procesamiento de señales para aprendizaje automático

Este ejemplo funciona sobre el conjunto de datos utilizado para la práctica 2, en cuyo enunciado pueden verse los detalles de la estructura de este dataset.

## Imports

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import os

## Entendiendo los datos

Primero intenta cargar un solo fichero, por ejemplo la grabación 3 del participante 5. Después muestra las primeras 5 líneas.

In [ ]:
s_05_03 = pd.read_csv("dataset/data/sub_05/rec_3.csv")
s_05_03

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/data/sub_05/rec_3.csv'

### Correlación entre datos
Echad un ojo a cómo están correlacionados los datos entre sí, pero usad un número limitado de columnas para que podamos estudiar el gráfico adecuadamente (18, por ejemplo).

Hay un patrón curioso en los datos y es que todos las coordnadas en sus respectivos ejes están muy correlacionados entre sí. Aquí hay 2 posibles conclusiones:

* Asumir que hay información redundante y que podemos cargarnos muchas de las columnas.
* Entender lo que significan los datos y darse cuenta que la posición global de los puntos influye más que el movimiento relativo.

Os voy adelantando que es el segundo caso y que es posible eliminar esas correlaciones sin eliminar información. Para ello, la clave es restar la posición de la pelvis al del resto de posiciones, ya que esta representa la posición general del cuerpo.

In [ ]:
# Resta la posición de la pelvis al del resto de elementos
# y comprueba de nuevo cómo son las correlaciones
resta = s_05_03.copy()
#resta.iloc[:,3,-3:3] -= s_05_03.iloc[:,0:1].to_numpy()
#resta.iloc[:,4,-3:3] -= s_05_03.iloc[:,1:2].to_numpy()
#resta.iloc[:,5,-3:3] -= s_05_03.iloc[:,2:3].to_numpy()

for i in range(3)
  resta.iloc[:,1+3,-3:3] -= s_05_03.iloc[:,i:i+1].to_numpy()

#resta_x = s_05_03.iloc[:,3:-3:3] - s_05_03.iloc[:, 0:1].to_numpy()
#la muñeca y el codo están relacionados porque si mueves el codo mueves la muñeca

Como puede verse todavía hay elementos que están muy correlacionados entre sí. Eso se debe a que las posiciones de cada articulación del cuerpo depende de otra articulación previa que la mueve. Es posible eliminar todas esas correlaciones usando la misma estrategia que con la pelvis, pero para este caso, ya tenemos suficiente información. Aún así, os lo planteo como reto para hacer una limpieza más fina de los datos. Ahora podríamos plantear eliminar columnas muy correlacionadas entre sí, pero lo dejaremos para una fase posterior.

### Visualización de los datos
Hay muchos elementos interesantes a visualizar así, vamos a empezar con la trayectoria que sigue el movimiento general representado por la pelvis.

Ahora vamos a definir una función que nos permita graficar la posición tridimensional de los puntos a partir de los datos básicos previamente a la eliminación de las correlaciones. La definición de la función debería ser la siguiente:

*def plot_frame(data: pd.DataFrame, i_frame: int) -> (matplotlib.figure.Figure, matplotlib.axes.Axes)*

Nota: Los valores devuletos corresponden a una figura y unos ejes donde se pintará el resultado. Es conveniente evitar usar plt.show() en la función para luego poder combinar varias si lo necesitamos.

Ten en cuenta que las últimas 3 coordenadas corresponden a la velocidad y que no será relevante en esta gráfica. También considera que para visualizarlo adecuadamente necesitarás que la escala sea la misma en los 3 ejes. A partir de esta idea, se puede elaborar para graficar animaciones. El problema de las animaciones es que se tardan bastante en ver y no se puede visualizar del tirón varios segmentos.

In [ ]:
def plot_frame(data, i_frame = 0):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')

    return fig, ax

fig, ax =  plot_frame(s_05_03, 130)
plt.show()

Vale, ese gráfico está muy bien, pero no resulta muy útil para ver cómo se ha movido exactamente una articulación. Ahora intentaremos mostrar eso, también en 3D, pero en vez de enseñar una línea de puntos. Para ello será necesario definir la siguiente función:

*def plot_joint(data: pd.DataFrame, i_joint: int) -> (matplotlib.figure.Figure, matplotlib.axes.Axes)*

In [ ]:
def plot_joint(data, joint_i):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')

    return fig, ax



## Ventaneo
Una normalización importante de los datos, es asegurarse que todos tengan el mismo formato. En este caso pretendemos que se separen en segmentos con la misma duración, específicamente de 4 segundos. Para poder aplicarlo apropiadamente a otras grabaciones, define una función que haga este ventaneo. Piensa en qué parámetros debería recibir.

In [ ]:
freq =60
segundos = 4
ventana = freq*segundos
a=[]
for i in range(0,len(resta)-ventana+1, ventana):
  a.append(resta.iloc[i:i+ventana])

## Extracción de características
Una vez están definidas las ventanas, el siguiente paso consiste en calcular métricas que puedan resultarnos de utilidad para resumir la señal. Existen muchas muy útiles que os invito a buscar, aunque para este ejemplo queremos las siguientes:

* Media
* Desviación típica
* [Energía de la señal](https://en.wikipedia.org/wiki/Energy_(signal_processing)

Es importante tener en cuenta sobre qué eje hay que aplicar las fórmulas.

Con esto, ya tendríamos preparadas las piezas básicas para montar el puzzle, solo nos queda llevar a cabo la carga para cada participante.

## Carga y combinación de ficheros
Ahora carga todos los ficheros y combínalos en uno solo manteniendo la información sobre a qué participante grabación corresponden. Te recomiendo investigar las funciones de la librería os que incluye Python para ver funciones que listen automáticamente los ficheros de una carpeta.

### Normalización por participante
Esto es importante hacerlo para descartar el efecto de las diferentes dimensiones de cada persona en los resultados, ya que lo que queremos hacer es estudiar los tipos de movimiento como tal.

## Juntar todo
El último paso será utilizar todos los conocimientos desarrollados durante este notebook y combinarlos para realizar el prcoesamiento previo necesario para la segunda práctica.